In [12]:
import os
import pyodbc
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_I_PORTFOLIOPOS_S;\
                       Trusted_Connection=yes;')

In [15]:
sql_query_monthly = """
select
C.*
from
(
select
pfm.Aggrid,
pfm.postdt,
pfm.rtntyp,
pfm.totrtn1mo,
pfm.totrtn3mo,
pfm.totrtn6mo,
pfm.totrtn12mo,
mars.PortfolioUID,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountUID,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select 
aggrid,
postdt,
rtntyp,
totrtn1mo,
totrtn3mo,
totrtn6mo,
totrtn12mo
from IM_I_IDWBRRESULTS_S.dbo.MA_PP_PFMHIST
where incltyp = 'I' and crncyid = 1
) as pfm
  inner join
    (
	select
	A.AGGRID,
	A.PortfolioUID
	from
		(
		select
		AGGRID,
		PortfolioUID,
		DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		) as A
		inner join
		(
		select
		AGGRID,
		max(DWUpdateDate) as max_DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		group by AGGRID
		) as B
		on A.AGGRID=B.AGGRID and A.DWUpdateDate=B.max_DWUpdateDate
	) as mars
  on pfm.AGGRID=mars.aggrid
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on mars.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate
  
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_RP as RP
  on cg.portfoliouid =  RP.RPPortfolioUID and pfm.postdt between  RP.dweffectivefromdate and RP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'RP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on cg.portfoliouid =  GM.GMPortfolioUID and pfm.postdt between  GM.dweffectivefromdate and GM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on cg.portfoliouid =  GP.GPPortfolioUID and pfm.postdt between  GP.dweffectivefromdate and GP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MR as MR
  --on cg.portfoliouid =  MR.MRPortfolioUID and pfm.postdt between  MR.dweffectivefromdate and MR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'MR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on cg.portfoliouid =  SG.SGPortfolioUID and pfm.postdt between  SG.dweffectivefromdate and SG.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'SG'
  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='RP' THEN RP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='MR' THEN MR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='SG' THEN SG.InvestmentAccountUID

								end
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
) as C
"""
# Save the data 
df_monthly = pd.read_sql(sql_query_monthly, cnxn)

In [17]:
sql_query_daily = """
select
C.*
from
(
select
pfm.Aggrid,
pfm.postdt,
pfm.rtntyp,
mars.PortfolioUID,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountUID,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select
aggrid,
'D' as rtntyp,
DATEADD(m, DATEDIFF(m, 0, postdt), 0) as postdt
from IM_I_IDWBRRESULTS_S.dbo.MA_PP_PFHIST
where INCLTYP = 'I' and TOTRTNDAY is not NULL  and CRNCYID = 1
group by aggrid, DATEADD(m, DATEDIFF(m, 0, postdt), 0)
having sum(1) >= 15
) as pfm
  inner join
    (
	select
	A.AGGRID,
	A.PortfolioUID
	from
		(
		select
		AGGRID,
		PortfolioUID,
		DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		) as A
		inner join
		(
		select
		AGGRID,
		max(DWUpdateDate) as max_DWUpdateDate
		from IM_I_PORTFOLIOPOS_S.dbo.MarsAggrMapping
		group by AGGRID
		) as B
		on A.AGGRID=B.AGGRID and A.DWUpdateDate=B.max_DWUpdateDate
	) as mars
  on pfm.AGGRID=mars.aggrid
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on mars.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate
  
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_RP as RP
  on cg.portfoliouid =  RP.RPPortfolioUID and pfm.postdt between  RP.dweffectivefromdate and RP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'RP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GM as GM
  --on cg.portfoliouid =  GM.GMPortfolioUID and pfm.postdt between  GM.dweffectivefromdate and GM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GM'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GP as GP
  --on cg.portfoliouid =  GP.GPPortfolioUID and pfm.postdt between  GP.dweffectivefromdate and GP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_MR as MR
  --on cg.portfoliouid =  MR.MRPortfolioUID and pfm.postdt between  MR.dweffectivefromdate and MR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'MR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SG as SG
  --on cg.portfoliouid =  SG.SGPortfolioUID and pfm.postdt between  SG.dweffectivefromdate and SG.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'SG'
  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='RP' THEN RP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GM' THEN GM.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='GP' THEN GP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='MR' THEN MR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='SG' THEN SG.InvestmentAccountUID

								end
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
) as C
"""
# Save the data 
df_daily = pd.read_sql(sql_query_daily, cnxn)
    
# Close the connection with LASR
cnxn.close()

In [16]:
print(df_monthly.shape)
df_monthly.head()

(17720751, 15)


,Aggrid,postdt,rtntyp,totrtn1mo,totrtn3mo,totrtn6mo,totrtn12mo,PortfolioUID,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,29775872,2015-02-27,M,99.358528,100.798650,101.895527,102.356700,37218,AM,8065092.0,AG,None,None,None,None
1,34198984,2004-08-31,M,0.000000,100.000000,100.000000,100.000000,42620,AM,43002587.0,AG,None,None,None,None
2,95302619,2017-09-29,M,NaN,NaN,NaN,NaN,119469,AM,7101555.0,SA,IA,None,None,None
3,67053161,2017-12-29,M,104.322740,108.708284,116.261590,141.284785,83693,GA,90001150.0,SA,IA,None,None,None
4,162803565,2018-09-28,M,100.104757,102.719003,102.445233,102.445233,319545,AT,32075504.0,SA,PCS,None,None,None


In [18]:
print(df_daily.shape)
df_daily.head()

(9286942, 11)


,Aggrid,postdt,rtntyp,PortfolioUID,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,26056088,2016-12-01,D,32455,AM,12000012.0,FUND,None,MF,AF,NG
1,45792848,2016-10-01,D,297010,AA,30096702.0,UNAVAILABLE,None,None,None,None
2,57213211,2004-11-01,D,71508,AM,3012723.0,AG,None,None,None,None
3,74726521,2017-01-01,D,93437,AT,9014637.0,UNAVAILABLE,None,None,None,None
4,43311895,2016-12-01,D,54295,AM,12000012.0,FUND,None,MF,AF,NG


In [20]:
def align_values(row):
    if row['rtntyp'] == 'M':
        return row['totrtn1mo']
    elif row['rtntyp'] == 'Q':
        return row['totrtn3mo']
    elif row['rtntyp'] == 'S':
        return row['totrtn6mo']
    elif row['rtntyp'] == 'A':
        return row['totrtn12mo']
    else:
        return np.nan
    
df_monthly['rtntyp_values'] = df_monthly.apply(lambda x: align_values(x), axis=1)
df_monthly = df_monthly[['aggrid', 'postdt', 'rtntyp', 'rtntyp_values']]
# Drop rows with NULL values
df_monthly = df_monthly.dropna(axis=0)
print(df_monthly.shape)
df_monthly.head()

KeyboardInterrupt: 

In [ ]:
df_monthly.drop(['rtntyp_values'], axis=1, inplace=True)
df_monthly.head()

In [ ]:
new_aggrid_list = []
new_date_list = []
new_rtntyp_list = []
new_hier_list = []
new_invport_list = [] #InvestmentPortfolioTypeCode,
new_lab1 = [] #InvestmentAccountTypeCode,
new_lab2 = [] #SeparateAccountTypeCode,
new_lab3 = [] #FundAccountTypeCode,
new_lab4 = [] #FundAccountSubTypeCode,
new_lab5 = [] #FundAccountGroupTypeCode

for aggrid, postdt, rtntyp in df_monthly.itertuples(index=False):
    if rtntyp == 'A':
        for d in pd.date_range(end=postdt, periods=12, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(4)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'S':
        for d in pd.date_range(end=postdt, periods=6, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(3)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'Q':
        for d in pd.date_range(end=postdt, periods=3, freq='MS'):
            new_aggrid_list.append(aggrid)
            new_date_list.append(d)
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(2)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)
    if rtntyp == 'M':
            new_aggrid_list.append(aggrid)
            new_date_list.append(postdt.replace(day=1))
            new_rtntyp_list.append(rtntyp)
            new_hier_list.append(1)
            
            new_invport_list.append(InvestmentPortfolioTypeCode)
            new_lab1.append(InvestmentAccountTypeCode)
            new_lab2.append(SeparateAccountTypeCode)
            new_lab3.append(FundAccountTypeCode)
            new_lab4.append(FundAccountSubTypeCode)
            new_lab5.append(FundAccountGroupTypeCode)

df_monthly_1stm = pd.DataFrame(columns=['aggrid', 'postdt', 'rtntyp'])
df_monthly_1stm['aggrid'] = new_aggrid_list
df_monthly_1stm['postdt'] = new_date_list
df_monthly_1stm['rtntyp'] = new_rtntyp_list
df_monthly_1stm['hier'] = new_hier_list

#new fields
df_monthly_1stm['PortType'] = new_invport_list
df_monthly_1stm['InvestmentAccountTypeCode'] = new_lab1
df_monthly_1stm['SeparateAccountTypeCode'] = new_lab2
df_monthly_1stm['FundAccountTypeCode'] = new_lab3
df_monthly_1stm['FundAccountSubTypeCode'] = new_lab4
df_monthly_1stm['FundAccountGroupTypeCode'] = new_lab5


print(df_monthly_1stm.shape)
df_monthly_1stm.head()

In [ ]:
#check no duplicate records
print(df_monthly_1stm.drop_duplicates(subset=['aggrid', 'postdt'], keep='first', inplace=False).count())
#there are a small few - remove them
df_monthly_1stm.drop_duplicates(subset=['aggrid', 'postdt'], keep='first', inplace=True)
print(df_monthly_1stm.shape)
df_monthly_1stm.head()

In [ ]:
df_daily.rename(columns={'postdt_m': 'postdt'}, inplace=True)
df_daily['hier']=0
df_daily.head()

# COMBINE THEM

In [ ]:
df_combo = df_monthly_1stm.append(df_daily, ignore_index=True)
print(df_combo.shape)
df_combo.head()

In [ ]:
df_left = df_combo.groupby(['aggrid', 'postdt'])['hier'].min().reset_index()
print(df_left.shape)

df_final = df_left.merge(df_combo, left_on=['aggrid', 'postdt', 'hier'], right_on=['aggrid', 'postdt', 'hier'], how='left')

df_final.drop(['hier'], axis=1, inplace=True)
print(df_final.shape)
df_final.head()

In [ ]:
df_summary_out = df_final.groupby(['postdt','rtntyp'])['aggrid'].count().reset_index()
df_summary_out.rename(columns={'aggrid': 'aggrid_cnt'}, inplace=True)
print(df_summary_out.shape)
df_summary_out.head()

In [ ]:
#output the summarized version
df_summary_out.to_csv('PF_Daily_n_Monthly_w_Labels_20181004.csv')